# Distal Events Table

__Description__: Excess rainfall is calculated for ...
    
__Input__: The metadata *.json* file from either `EventsTable.ipynb` or `PM-EventsTable.ipynb`, which contains the final precipitation curves for each randomized event.

__Output__:
    
---

## A. Load Libraries, Parameters, and Data:

### Libraries:

In [1]:
import sys
sys.path.append('core')
from hydromet import*

### Parameters:

#### Papermill (site specific):

In [2]:
## Excess rainfall parameters:
CN = 85       # Curve number


## Filenames and paths:
AOI = 'DC'     # Area of interest
Model = 'P01'  # Model name

root_dir = pl.Path(os.getcwd())
outputs_dir = root_dir/'Outputs'
datarepository_dir = root_dir/'DataRepository'


## Options:
display_print = True

##### Convert all paths to objects:

In [3]:
outputs_dir = pl.Path(outputs_dir)
datarepository_dir = pl.Path(datarepository_dir)

#### Global (project specific):

In [4]:
## Filenames and paths:
CN_Distribution = 'NEH630_Table_10_1.json'


## Options:
adjust_CN_less24 = False # Set to True to adjust the curve number when the storm duration is less than 24 hours

start_time = time.time()

###  Data:

#### Metadata from EventsTable.ipynb:

In [5]:
with open(outputs_dir/'Metadata_{0}_{1}.json'.format(AOI, Model)) as f:
    md = json.load(f)

##### Extract the durations:

In [6]:
durations = list(md.keys())
print('Durations:', durations)

Durations: ['H06', 'H12', 'H24', 'H96']


##### Extract the boundary condition names:

In [7]:
BCN = list(md[durations[0]]['BCName'].keys())
print('Boundary Condition Names:', BCN)

Boundary Condition Names: ['D01']


#### Curve number distribution:

In [8]:
arc_data = get_CN_distribution(datarepository_dir, CN_Distribution, CN, display_print)

{'Dry': 70, 'Wet': 94}


---

## B. TBD

In [ ]:
for dur in durations:
    for name in BCN:

In [9]:
dur = durations[0]
idur = int(dur.replace('H', ''))
name = BCN[0]
ID = 'E60001'
i = 0

outfiles = []

scen = md[dur]['BCName'][name]

groups = scen['groups']

precip = scen['precip']
sim_ID = list(precip.keys())
n_rand_events = len(sim_ID)

#params = scen['parameters']
#seed = params['seed']
#tempEpsilon = params['tempEpsilon']
#tempEpsilon2 = params['tempEpsilon2']
#convEpsilon = params['convEpsilon']
#volEpsilon = params['volEpsilon']
seed = 2149
tempEpsilon = 4 
tempEpsilon2 = 2 
convEpsilon = 100  
volEpsilon = 67  


## Calculate random curve numbers:
df_CN = prep_cn_table(CN, arc_data) 
fitted_cn = find_optimal_curve_std(df_CN, 'Lower', 'Upper')
filename = "Rand_CN_distal_{0}_Dur{1}_tempE{2}_convE{3}_volE{4}_Se{5}.csv".format(AOI, idur, tempEpsilon, convEpsilon, volEpsilon, seed)
outfiles.append(filename)
random_cns = RandomizeData(fitted_cn, n_rand_events, outputs_dir, filename, seed=seed, variable='CN', lower='Lower', upper='Upper', display_print = display_print)

In [9]:
runids = []

for i, ID in enumerate(sim_ID):
    excess = []
    cum_precip = list(precip[ID].values())
    total_precip = cum_precip[-1]
    cn = random_cns.iloc[i]['Random CN']
    if idur < 24 and adjust_CN_less24:
        adj_cn, s, ia = update_CN(cn, idur, total_precip)
    else:
        s  = S_24hr(cn)
        ia = IA_24hr(s)
    for p in cum_precip:
        excess.append(calculate_excess(p, ia, s))
    runid='E{}_'.format(event)+'{}Hr_'.format(dur)+'Q{}_'.format(int(orig_q))+'D{}_'.format(decile)+'CN{}'.format(cn)


## C. Combine Temporal Curves:

---

## D. Save the Results:

### Combined excess rainfall curves:

### All metadata:

---

# End